# 준비물

## module 모음

In [ ]:
import requests
from selenium import webdriver
from bs4 import BeautifulSoup

import time
import random

import pickle

import pandas as pd

## url모음

In [ ]:
urls=["https://finance.naver.com/sise/sise_market_sum.nhn?&page="+str(x) for x in range(1,11)]

# 타겟 리스트 정리하기

## 시총순위, 종목코드 가져오기

In [ ]:
soup_dict={}

for j,url in enumerate(urls):
    
    page = requests.get(url)

    page_html = page.text

    page_soup = BeautifulSoup(page_html, 'html.parser')
    
    atags=page_soup.find_all('a','tltle')
    
    for i,atag in enumerate(atags):
        title=atag.get_text()
        if title not in soup_dict:
            soup_dict[title]=[(j*50)+i+1,atag.get_attribute_list('href')[0][-6:]]
        else:
            soup_dict[title+"2"]=[(j*50)+i+1,atag.get_attribute_list('href')[0][-6:]]
    time.sleep(random.randint(500,2500)*0.001)

In [ ]:
soup_dict

## 타겟URL생성

In [ ]:
for x in soup_dict:
    soup_dict[x].append("https://navercomp.wisereport.co.kr/v2/company/c1010001.aspx?cmp_cd="+soup_dict[x][1])

In [ ]:
soup_dict

> 임시저장단계  
종목명/시총순위/종목코드/타겟URL

In [ ]:
with open('step_1_done.pickle', 'wb') as handle:
    pickle.dump(soup_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

- 유사시 LOAD가능

In [ ]:
with open('step_1_done.pickle', 'rb') as handle:
    test_dict=pickle.load(handle)
soup_dict=test_dict.copy()

-------------------

# 타겟URL을 통한 크로울링

## 웹드라이버 가동(백그라운드)

In [ ]:
chrome_options = webdriver.ChromeOptions()  # 크롬 옵션 객체 생성
chrome_options.add_argument('headless')  # headless 모드 설정
chrome_options.add_argument("--disable-gpu")  # gpu 사용 안하도록 설정
chrome_options.add_argument("lang=ko_KR")  # 한국어로 실행되도록 설정
driver = webdriver.Chrome('C:/Chrome Driver/chromedriver.exe', chrome_options=chrome_options)

## 타겟 URL이동 및 타겟 ELEMENT접속

In [ ]:
# 특이종목 삭제용 리스트
drop_list=[]

In [ ]:
for x in soup_dict:
    #URL접속
    driver.implicitly_wait(3)
    driver.get(soup_dict[x][2])
    time.sleep(1)
    driver.implicitly_wait(3)
    #xpath로 가져올 데이터(테이블) 특정.
    for i in [1,5,8,9,10,20,21,22,23,24,26,27,28,29,31,32]:
        try:
            for j in [3,6,7]:
                a=driver.find_element_by_xpath('/html/body/div/form/div[1]/div/div[2]/div[3]/div/div/div[14]/table[2]/tbody/tr[{}]/td[{}]'.format(i,j))
                soup_dict[x].append(a.text)
        except:
            drop_list.append(x)
            print(x)
            break
    print(soup_dict[x])

### 특이 종목 삭제

In [ ]:
for drop_item in drop_list:
    soup_dict.pop(drop_item)

> 임시저장단계  
step_2까지 완료.

In [ ]:
with open('step_2_done.pickle', 'wb') as handle:
    pickle.dump(soup_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

# 가져온데이터로 기초 DataFrame생성

## 가공할 것 가져오기

In [ ]:
# 가공할 인덱스모음
for_indexs=[]

In [ ]:
#인덱스 가져오기
driver.get(soup_dict['삼성전자'][2])
time.sleep(1)
driver.implicitly_wait(3)
for i in range(1,33):
    a=driver.find_element_by_xpath('/html/body/div/form/div[1]/div/div[2]/div[3]/div/div/div[14]/table[2]/tbody/tr[{}]/th'.format(i))
    for_indexs.append(a.text)

## 가져온 인덱스 가공하기

In [ ]:
indexs=['순위','종목코드','href']
add_texts=['[2019]','[2019.12]','[2020.03]']
for i in add_texts:
    for index in for_indexs:
        indexs.append(index+i)

In [ ]:
df=pd.DataFrame(soup_dict)

In [ ]:
df.index=indexs

In [ ]:
df2=df.T

In [ ]:
df2.to_excel('the_naver1.xlsx')

In [ ]:
df2

# 종료 알림